# SkillFactory
## Введение в ML, введение в sklearn

В этом задании мы с вами рассмотрим данные с конкурса [Задача предсказания отклика клиентов ОТП Банка](http://www.machinelearning.ru/wiki/index.php?title=%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0_%D0%BF%D1%80%D0%B5%D0%B4%D1%81%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D0%B8%D1%8F_%D0%BE%D1%82%D0%BA%D0%BB%D0%B8%D0%BA%D0%B0_%D0%BA%D0%BB%D0%B8%D0%B5%D0%BD%D1%82%D0%BE%D0%B2_%D0%9E%D0%A2%D0%9F_%D0%91%D0%B0%D0%BD%D0%BA%D0%B0_%28%D0%BA%D0%BE%D0%BD%D0%BA%D1%83%D1%80%D1%81%29)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

### Грузим данные

Считаем описание данных

In [2]:

df_descr = pd.read_csv('data/otp_description.csv', sep='\t', encoding='utf8')

In [52]:
df['SOCSTATUS_PENS_FL'].unique()

array([0, 1])

Считаем обучающую выборки и тестовую (которую мы как бы не видим)

In [4]:
df_train = pd.read_csv('data/otp_train.csv', sep='\t', encoding='utf8')

In [5]:
df_train.shape

(15223, 52)

In [6]:
df_test = pd.read_csv('data/otp_test.csv', sep='\t', encoding='utf8')

In [7]:
df_test.shape

(14910, 52)

In [8]:
df_train.head()

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,...,REG_PHONE_FL,GEN_PHONE_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED
0,59910150,0,49,1,0,1,2,1,Среднее специальное,Состою в браке,...,0,1,1,1,6,2,1,1580,1580,NaN
1,59910230,0,32,1,0,1,3,3,Среднее,Состою в браке,...,0,1,1,1,6,1,1,4020,4020,NaN
2,59910525,0,52,1,0,1,4,0,Неполное среднее,Состою в браке,...,0,1,2,1,11,0,0,0,0,NaN
3,59910803,0,39,1,0,1,1,1,Высшее,Состою в браке,...,1,1,1,1,6,3,1,"1589,92333333333",1590,NaN
4,59911781,0,30,1,0,0,0,0,Среднее,Состою в браке,...,0,1,2,1,16,2,1,"1152,15",2230,NaN


## Объединим две выборки

Так как пока мы пока не умеем работать sklearn  Pipeline, то для того, чтобы после предобработки столбцы в двух выборках находились на своих местах.

Для того, чтобы в дальнейшем отделить их введем новый столбец "sample"

In [9]:
df_train.loc[:, 'sample'] = 'train'
df_test.loc[:, 'sample'] = 'test'

In [10]:
df = df_test.append(df_train).reset_index(drop=True)

In [11]:
df.shape

(30133, 53)

### Чуть-чуть посмотрим на данные

Посмотрим типы данных и их заполняемость

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30133 entries, 0 to 30132
Data columns (total 53 columns):
AGREEMENT_RK                  30133 non-null int64
TARGET                        30133 non-null int64
AGE                           30133 non-null int64
SOCSTATUS_WORK_FL             30133 non-null int64
SOCSTATUS_PENS_FL             30133 non-null int64
GENDER                        30133 non-null int64
CHILD_TOTAL                   30133 non-null int64
DEPENDANTS                    30133 non-null int64
EDUCATION                     30133 non-null object
MARITAL_STATUS                30133 non-null object
GEN_INDUSTRY                  27420 non-null object
GEN_TITLE                     27420 non-null object
ORG_TP_STATE                  27420 non-null object
ORG_TP_FCAPITAL               27425 non-null object
JOB_DIR                       27420 non-null object
FAMILY_INCOME                 30133 non-null object
PERSONAL_INCOME               30133 non-null object
REG_ADDRESS_PRO

Видим, что часть данных - object, скорее всего стоки.


Давайте выведем эти значения для каждого столбца

In [59]:
for i in df_train.columns: # перебираем все столбцы
    if str(df_train[i].dtype) == 'object': # если тип столбца - object
        print('='*10)
        print(i) # выводим название столбца
        print(df_train[i].unique()[:5]) # выводим все его значения (но делаем set - чтоб значения не повторялись)
        print('\n') # выводим пустую строку

EDUCATION
['Среднее специальное' 'Среднее' 'Неполное среднее' 'Высшее'
 'Неоконченное высшее']


MARITAL_STATUS
['Состою в браке' 'Гражданский брак' 'Разведен(а)' 'Не состоял в браке'
 'Вдовец/Вдова']


GEN_INDUSTRY
['Торговля' 'Информационные технологии' 'Образование'
 'Государственная служба' 'Другие сферы']


GEN_TITLE
['Рабочий' 'Специалист' 'Руководитель среднего звена'
 'Руководитель высшего звена' 'Служащий']


ORG_TP_STATE
['Частная компания' 'Индивидуальный предприниматель'
 'Государственная комп./учреж.' nan 'Некоммерческая организация']


ORG_TP_FCAPITAL
['Без участия' nan 'С участием']


JOB_DIR
['Вспомогательный техперсонал' 'Участие в основ. деятельности'
 'Адм-хоз. и трансп. службы' 'Пр-техн. обесп. и телеком.'
 'Служба безопасности']


FAMILY_INCOME
['от 10000 до 20000 руб.' 'от 20000 до 50000 руб.' 'свыше 50000 руб.'
 'от 5000 до 10000 руб.' 'до 5000 руб.']


PERSONAL_INCOME
['5000' '12000' '9000' '25000' '15000']


REG_ADDRESS_PROVINCE
['Оренбургская область' 'Кабарди

Mожно заметить что некоторые переменные, которые обозначены как строки (например PERSONAL_INCOME) на самом деле числа, но по какой-то причине были распознаны как строки

Причина же что использовалась запятая для разделения не целой части числа..

Перекодировать их можно например так:

In [14]:
df['PERSONAL_INCOME'].map(lambda x: x.replace(',', '.')).astype('float')

0         7000.0
1         4100.0
2        40000.0
3        20000.0
4        17000.0
5         7000.0
6        10000.0
7         7500.0
8         9000.0
9         4000.0
10       15000.0
11       12000.0
12        8000.0
13       20000.0
14       18500.0
15       14000.0
16        6000.0
17        9000.0
18        5000.0
19       15000.0
20        6100.0
21       22180.0
22        9000.0
23        8700.0
24        7000.0
25        6000.0
26       15000.0
27       12000.0
28       11000.0
29        8000.0
          ...   
30103    12000.0
30104    10000.0
30105    10000.0
30106     8000.0
30107    11000.0
30108     7000.0
30109    10000.0
30110     8000.0
30111    25000.0
30112    10000.0
30113    12000.0
30114     7000.0
30115    12000.0
30116    14500.0
30117    12000.0
30118     8000.0
30119     7000.0
30120    10000.0
30121    10000.0
30122    10000.0
30123    15000.0
30124     8100.0
30125     9500.0
30126    13000.0
30127    10000.0
30128    12000.0
30129    10000.0
30130     6000

Такой эффект наблюдается в столбцах `PERSONAL_INCOME`, `CREDIT`, `FST_PAYMENT`, `LOAN_AVG_DLQ_AMT`, `LOAN_MAX_DLQ_AMT`

### Теперь ваше небольшое исследование

#### Задание 1. Есть ли пропуски в данных? Что с ними сделать?

(единственного верного ответа нет - аргументируйте)

In [15]:
# Посмотрим количество записей в датафрейме
number_of_records = df.shape[0] # он же len(df)
print(f"Всего записей:\t{number_of_records}")

# Пробежимся по всем столбцам, чтобы посмотреть, где пропуски
for i in df.columns:
    number_of_i_records = df[i].dropna().count()
    if number_of_i_records < number_of_records:
        print(f"{i}:\t{number_of_i_records}({(number_of_i_records/number_of_records):.2%})")
        
# А вообще сделаем из нее функцию:
def missingData( dataFrame ):
    number_of_records = dataFrame.shape[0] # он же len(df)
    print(f"Всего записей:\t{number_of_records}")
    for i in dataFrame.columns:
        number_of_i_records = dataFrame[i].dropna().count()
        if number_of_i_records < number_of_records:
            print(f"{i}:\t{number_of_i_records}({(number_of_i_records/number_of_records):.2%})")

Всего записей:	30133
GEN_INDUSTRY:	27420(91.00%)
GEN_TITLE:	27420(91.00%)
ORG_TP_STATE:	27420(91.00%)
ORG_TP_FCAPITAL:	27425(91.01%)
JOB_DIR:	27420(91.00%)
TP_PROVINCE:	29543(98.04%)
REGION_NM:	30131(99.99%)
WORK_TIME:	27416(90.98%)
PREVIOUS_CARD_NUM_UTILIZED:	600(1.99%)


**PREVIOUS_CARD_NUM_UTILIZED** - из описания (если пусто - 0), поэтому это первое, что исправим.
Однозначно **fillna(0)**

In [16]:
df[ df['GEN_INDUSTRY'].isnull() ][ ['GEN_TITLE', 'ORG_TP_STATE', 'ORG_TP_FCAPITAL', 'JOB_DIR'] ].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2713 entries, 1 to 30131
Data columns (total 4 columns):
GEN_TITLE          0 non-null object
ORG_TP_STATE       0 non-null object
ORG_TP_FCAPITAL    6 non-null object
JOB_DIR            0 non-null object
dtypes: object(4)
memory usage: 106.0+ KB


Что касается:
- **GEN_INDUSTRY**
- **GEN_TITLE**
- **ORG_TP_STATE**
- **ORG_TP_FCAPITAL**
- **JOB_DIR**

Скорее всего, это логически связанные столбцы.
Там где нет данных в одном столбце, нет данных и в остальных, поэтому сразу оратим внимание на этот большой кусок.
Всего в датасете отсутствует менее 10% этих данных. Набор данных в каждом их них сложный, поэтому брать усреднение или вводить новый идентификатор типа "Другое" было бы некорректно.

**Вердикт: dropna()**

In [17]:
missingData( df[ ~df['GEN_INDUSTRY'].isnull() ] )

Всего записей:	27420
ORG_TP_FCAPITAL:	27419(100.00%)
TP_PROVINCE:	26846(97.91%)
REGION_NM:	27418(99.99%)
WORK_TIME:	27416(99.99%)
PREVIOUS_CARD_NUM_UTILIZED:	558(2.04%)


После "удаления" столбцов связанных с "местом работы", видим, что пропусков почти не осталось. **TP_PROVINCE** никак не усреднишь, поэтому однозначно dropna(). А в остальных столбцах отсутствует бувально единичные значения, дропаем!
##### Итого:

In [18]:
df_clean = df.fillna({'PREVIOUS_CARD_NUM_UTILIZED': 0}).dropna()
df_clean.info() #Смотрим,что пропусков не осталось
print(f"Осталось от начального датасета:\t{(len(df_clean)/len(df)):.2%} данных")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26840 entries, 0 to 30132
Data columns (total 53 columns):
AGREEMENT_RK                  26840 non-null int64
TARGET                        26840 non-null int64
AGE                           26840 non-null int64
SOCSTATUS_WORK_FL             26840 non-null int64
SOCSTATUS_PENS_FL             26840 non-null int64
GENDER                        26840 non-null int64
CHILD_TOTAL                   26840 non-null int64
DEPENDANTS                    26840 non-null int64
EDUCATION                     26840 non-null object
MARITAL_STATUS                26840 non-null object
GEN_INDUSTRY                  26840 non-null object
GEN_TITLE                     26840 non-null object
ORG_TP_STATE                  26840 non-null object
ORG_TP_FCAPITAL               26840 non-null object
JOB_DIR                       26840 non-null object
FAMILY_INCOME                 26840 non-null object
PERSONAL_INCOME               26840 non-null object
REG_ADDRESS_PRO

#### Задание 2. Есть ли категориальные признаки? Что с ними делать?

Категориальных много, но рассмотрим только строковые (хотя GENDER, например, тоже  в общем-то категориальный тип ):
- EDUCATION
- MARITAL_STATUS
- GEN_INDUSTRY
- GEN_TITLE
- ORG_TP_STATE
- ORG_TP_FCAPITAL
- JOB_DIR
- FAMILY_INCOME
- REG_ADDRESS_PROVINCE
- FACT_ADDRESS_PROVINCE
- POSTAL_ADDRESS_PROVINCE
- TP_PROVINCE
- REGION_NM


In [66]:
cat_list = [ 
    'EDUCATION', 
    'MARITAL_STATUS', 
    'GEN_INDUSTRY', 
    'GEN_TITLE', 
    'ORG_TP_STATE', 
    'ORG_TP_FCAPITAL', 
    'JOB_DIR',
    'FAMILY_INCOME',
    'REG_ADDRESS_PROVINCE',
    'FACT_ADDRESS_PROVINCE',
    'POSTAL_ADDRESS_PROVINCE',
    'TP_PROVINCE',
    'REGION_NM'
]

# 
df_clean[ cat_list ] = df_clean[ cat_list ].astype('category')
df['EDUCATION'].astype(int)

ValueError: invalid literal for int() with base 10: 'Среднее специальное'

#### Задание 3. Фунция предобработки

Напишите функцию, которая бы

* Удаляло идентификатор `AGREEMENT_RK`
* Избавлялась от проблем с '.' и ',' в стобцах PERSONAL_INCOME, CREDIT, FST_PAYMENT, LOAN_AVG_DLQ_AMT, LOAN_MAX_DLQ_AMT
* Что-то делала с пропусками
* Кодировала категориальные признаки

В результате, ваш датафрейм должен содержать только числа и не содержать пропусков!

In [47]:
def preproc_data(df_input):
    df_output = df_input.copy()
    
    # Удаляло идентификатор AGREEMENT_RK
    df_output.drop('AGREEMENT_RK', axis=1, inplace=True)
    
    # Избавлялась от проблем с '.' и ',' в стобцах PERSONAL_INCOME, CREDIT, FST_PAYMENT, LOAN_AVG_DLQ_AMT, LOAN_MAX_DLQ_AMT
    float_list = ['PERSONAL_INCOME', 'CREDIT', 'FST_PAYMENT', 'LOAN_AVG_DLQ_AMT', 'LOAN_MAX_DLQ_AMT']
    df_output[ float_list ] = df_output[ float_list ].applymap(lambda x: x.replace(',', '.')).astype('float')
    
    
    return df_output

In [49]:
df_preproc = df.pipe(preproc_data)

df_train_preproc = df_preproc.query('sample == "train"').drop(['sample'], axis=1)
df_test_preproc = df_preproc.query('sample == "test"').drop(['sample'], axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30133 entries, 0 to 30132
Data columns (total 52 columns):
TARGET                        30133 non-null int64
AGE                           30133 non-null int64
SOCSTATUS_WORK_FL             30133 non-null int64
SOCSTATUS_PENS_FL             30133 non-null int64
GENDER                        30133 non-null int64
CHILD_TOTAL                   30133 non-null int64
DEPENDANTS                    30133 non-null int64
EDUCATION                     30133 non-null object
MARITAL_STATUS                30133 non-null object
GEN_INDUSTRY                  27420 non-null object
GEN_TITLE                     27420 non-null object
ORG_TP_STATE                  27420 non-null object
ORG_TP_FCAPITAL               27425 non-null object
JOB_DIR                       27420 non-null object
FAMILY_INCOME                 30133 non-null object
PERSONAL_INCOME               30133 non-null float64
REG_ADDRESS_PROVINCE          30133 non-null object
FACT_ADDRESS_

#### Задание 4. Отделите целевую переменную и остальные признаки

Должно получится:
* 2 матрицы: X и X_test
* 2 вектора: y и y_test

#### Задание 5. Обучение и оценка качества разных моделей

In [22]:
from sklearn.model_selection import train_test_split
# test_size=0.3, random_state=42

## Your Code Here


In [23]:
# Попробовать следующие "черные ящики": интерфейс одинаковый 
# Постепенно мы узнаем, что они делают а сейчас учимся понимать какой работает качественнее
#     fit, 
#     predict, 
#     predict_proba

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

## Your Code Here

In [24]:
# Посчитать метрики стандартные
# accuracy, precision, recall

from sklearn.metrics import accuracy_score, precision_score, recall_score

## Your Code Here

In [25]:
# Визуалищировать эти метрики всех моделей на одном графике (чтоб визуально сравнить)
# Возможно вас удивит качество! Но задача подобрана специально ;) Такое качество тоже бывает

## Your Code Here

In [26]:
# Потроить roc-кривые всех можелей на одном графике
# Вывести roc_auc каждой моделе
# Подпишите оси и линии

## Your Code Here

In [30]:
from sklearn.model_selection import cross_val_score
# Сделать k-fold (10 фолдов) кросс-валидацию каждой модели
# И посчитать средний roc_auc

## Your Code Here

In [31]:
# Взять лучшую модель и сделать предсказания (с вероятностями (!!!)) для test выборки

predict = 
## Your Code Here

SyntaxError: invalid syntax (<ipython-input-31-32ceb4195326>, line 3)

In [32]:
# Померить roc_auc на тесте
# Вывести текстом и на графике =)

## Your Code Here

### Больше обязательных заданий нет, но ниже вы можете провести исследование, поиграться с данными
### Это возможность написать код, который я посмотрю и в случае чего откомметирую. ;)
### Это не оценивается и остается на ваше усмотрение. Просто дополнительная возможность

In [ ]:
### И далее ;)